# Promises

A Promise is a placeholder for a value which isn't yet known. Julia's Tasks provide everything Promises provide though they are also generators which makes me feel uncomfortable about building abstractions on top of them that treat them as simple values. And RemoteRefs can be viewed as a form of Promise but don't have a nice API for the simple case since they are also intended to enable ongoing communication channels. Some people [really like](http://swannodette.github.io/2013/08/23/make-no-promises/) these sorts of bonus features. They seem to want a sort of swiss army knife for all async programming patterns. But to me it looks a bit like a bartender using a multi-tool to open bottles for the odd time he needs to cut a lemon. Most of the time when you are doing things asynchronously your really wish you were doing it synchronously. Promises are designed allow to think synchronously about things which could happen at any time. That is all.

In [1]:
@require "Promises" need Result @defer @thread;

If you have an expensive computation which isn't always needed a simple optimisation is to wrap it in a Deferred. This will prevent the computation from being executed until its actually needed.

In [2]:
@defer JSON.parse(readall("dependencies.json"))

Computation{Any}((anonymous function),:pending,#undef,#undef)

Then to unwrap it just call `need` on it

In [3]:
need(Out[2])

Dict{String,Any} with 1 entry:
  "development" => ["jest"=>"http://github.com/jkroso/jest/tarball/804af51"]

In the above example we used a promise to represent the parsed version of a JSON file because we weren't sure if it would actually be used or not. But when it is needed it takes a long time to compute because of the IO operation to read from the file system. In some scenarios the IO delay may be more of a concern. To handle this we could do the waiting in another thread leaving the current thread to continue with other things. i.e we reduce the overall time of the computation by doing steps in parrallel. But how does this secondary thread send the result of reading the file back to the main thread.

In [4]:
@thread JSON.parse(readall("dependencies.json"))

Result{Any}(Condition({}),:evaled,["development"=>["jest"=>"http://github.com/jkroso/jest/tarball/804af51"]],#undef)

In [5]:
need(Out[4])

Dict{String,Any} with 1 entry:
  "development" => ["jest"=>"http://github.com/jkroso/jest/tarball/804af51"]